In [1]:
import pandas as pd
import numpy as np
import os
import sys
import json
import requests
import gc
from tqdm import tqdm
from datetime import datetime, timedelta
import openpyxl

pd.options.display.max_columns = 100
pd.options.display.max_rows=1000

pd.options.display.max_columns = 100
pd.options.display.max_rows=1000


In [6]:
acs = pd.read_csv('../../cp_data.csv')['id'][1]
scr = pd.read_csv('../../cp_data.csv')['secret'][1]

In [7]:
report_type = 'cancels'
report_type = 'orders'

In [ ]:
## 날짜 페이징
import requests 
import hmac
import hashlib
from time import gmtime, strftime
from datetime import datetime, timedelta
import pandas as pd

REQUEST_METHOD = "GET"
DOMAIN = "https://api-gateway.coupang.com"
URL = f"/v2/providers/affiliate_open_api/apis/openapi/v1/reports/{report_type}"


def generateHmac(method, url, secretKey, accessKey, queryString=""):
    datetimeGMT = strftime('%y%m%d', gmtime()) + 'T' + strftime('%H%M%S', gmtime()) + 'Z'
    message = datetimeGMT + method + url + queryString
    signature = hmac.new(bytes(secretKey, 'utf-8'), message.encode('utf-8'), hashlib.sha256).hexdigest()
    return f"CEA algorithm=HmacSHA256, access-key={accessKey}, signed-date={datetimeGMT}, signature={signature}"

from tqdm import tqdm  # Import tqdm

def fetch_data(startDate, endDate, page):
    queryString = f"startDate={startDate}&endDate={endDate}&page={page}"
    fullURL = f"{DOMAIN}{URL}?{queryString}"
    authorization = generateHmac(REQUEST_METHOD, URL, SECRET_KEY, ACCESS_KEY, queryString)
    headers = {"Authorization": authorization, "Content-Type": "application/json"}
    response = requests.get(fullURL, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to fetch data: HT|TP {response.status_code} - {response.text}")
        return None
    
    try:
        return response.json()
    except ValueError:
        print(f"Failed to decode JSON from response: {response.text}")
        return None

def fetch_data_in_chunks(start_date, end_date):
    start = datetime.strptime(start_date, '%Y%m%d')
    end = datetime.strptime(end_date, '%Y%m%d')
    results = []

    date_range = (end - start).days // 30 + 1  # Calculate the number of chunks
    date_chunks = tqdm(range(date_range), desc="Fetching date ranges")

    for _ in date_chunks:
        chunk_end = start + timedelta(days=29)
        if chunk_end > end:
            chunk_end = end

        page = 0
        while True:
            response = fetch_data(start.strftime('%Y%m%d'), chunk_end.strftime('%Y%m%d'), page)
            if response:
                data = response.get('data', [])
                if data:
                    results.extend(data)
                    if len(data) < 1000:
                        break
                    page += 1
                else:
                    break
            else:
                break

        start = chunk_end + timedelta(days=1)
        if start > end:
            break

    return results

# Define your access keys and date range
ACCESS_KEY = acs
SECRET_KEY = scr
start_date = "20240301"
end_date = "20240509"

# Collect all data
all_data = fetch_data_in_chunks(start_date, end_date)
df = pd.DataFrame(all_data)

